In [ ]:
import os
import re
import sys
import git
import json
import tifffile
import skimage
import datetime
import numpy as np
import pandas as pd
from collections import namedtuple
from py4j.java_gateway import JavaGateway

In [ ]:
sys.path.insert(0, '../')
from dragonfly_automation import utils
from dragonfly_automation.gateway import gateway_utils
from dragonfly_automation.programs.pipeline_plate_program import PipelinePlateProgram

In [ ]:
# mode
ENV = 'dev'

# The root directory for the experiment
EXPERIMENT_DIR = 'D:/PR/dragonfly-automation-tests/20190823'

# The list of positions generated by the HCS Site Generator plugin
POSITION_LIST_FILENAME = '/Users/keith.cheveralls/Box/Coding/FileRename_forScope/INTERPOLATED_SLOPES_173304.pos'

In [ ]:
# parameters required for z-position interpolation
NUM_ROWS = 6
NUM_COLUMNS = 8
NUM_POSITIONS_PER_WELL = 25

CORNER_POSITIONS = (
    (7500,    7400,),
    (7450,    7350,)
)

In [ ]:
# interpolate the positions
# (note that the returned filename, `interpolated_position_list_filename`,
# is used after acquisition is complete to rename the TIFF stacks)
p = interpolated_position_list_filename = utils.interpolate_stage_positions(
    POSITION_LIST_FILENAME,
    (NUM_ROWS, NUM_COLUMNS),
    CORNER_POSITIONS,
    NUM_POSITIONS_PER_WELL)

In [ ]:
# data directory for dev mode
if ENV=='dev':
    EXPERIMENT_DIR = '../tests/output/ML0000_20190827'

In [ ]:
# autoreload only in 'dev' mode (because it doesn't work with py4j objects)
if ENV=='dev':
    %load_ext autoreload
    %autoreload 2

In [ ]:
# temporary hack to avoid a py4j Java error:
# create the datastore object here, rather than in program._initialize_datastore
datastore = None
datastore_dir = os.makedirs(os.path.join(EXPERIMENT_DIR, 'data'), exist_ok=True)

if ENV=='prod':
    # use wrap=False just to be safe
    gate, mm_studio, mm_core = gateway_utils.get_gate(env=ENV, wrap=False)
    datastore = mm_studio.data().createMultipageTIFFDatastore(datastore_dir, True, True)
    mm_studio.displays().createDisplay(datastore)

In [ ]:
# instantiate the program
program = PipelinePlateProgram(datastore, root_dir=EXPERIMENT_DIR, env=ENV, verbose=False)

In [ ]:
program.setup()

In [ ]:
program.run()

In [ ]:
# TODO: rename the TIFF stack filenames so that they include the imaging plate well ID,
# using the position list exported from the HCS Site Generator plugin
# Note that rename_tiff_files must be a method attribute of the program instance
# so that the file renaming can be logged, but it can be just a thin wrapper
# around a stand-alone renaming method in, e.g., dragonfly_automation.utils
program.rename_tiff_files(interpolated_position_list_filename)

# TODO: a second round of file renaming so that the filenames also include the 'real' well ID (and the target name?)